In [ ]:
!python --version
from google.colab import drive
drive.mount('/content/drive/')

Python 3.10.12
Mounted at /content/drive/


In [ ]:
import os
import numpy as np
from PIL import Image

In [ ]:
def find_golden_border(image_path):
    # 画像を開く
    image = Image.open(image_path)

    # 画像をNumPy配列に変換
    img_array = np.array(image)

    # 金色に近い色の範囲を定義 (RGB)
    lower_golden = np.array([180, 140, 0, 255])  # 追加された要素はアルファチャンネル
    upper_golden = np.array([255, 210, 100, 255])  # 追加された要素はアルファチャンネル

    # 金色に近い色のピクセルを検出
    mask = np.all(np.logical_and(img_array >= lower_golden, img_array <= upper_golden), axis=-1)

    # 検出された金色の領域をバウンディングボックスで取得
    coords = np.argwhere(mask)
    y0, x0 = coords.min(axis=0)
    y1, x1 = coords.max(axis=0)

    '''
    # 余白を追加して境界線の外側をトリミング
    padding = 0.5
    x0 = max(0, x0 - padding)
    y0 = max(0, y0 - padding)
    x1 = min(img_array.shape[1], x1 + padding)
    y1 = min(img_array.shape[0], y1 + padding)
    '''

    # 余白を追加して境界線の外側をトリミング by yamaT
    padding = 0.5
    x0 = max(0, x0 - padding + 1)
    y0 = max(0, y0 - padding + 2)
    x1 = min(img_array.shape[1], x1 + padding + 1)
    y1 = min(img_array.shape[0], y1 + padding)

    return (x0, y0, x1, y1)

In [ ]:
def trim_color_paper(image_path, output_path):
    # 金色の境界線を検出
    bbox = find_golden_border(image_path)

    # 画像を開く
    image = Image.open(image_path)

    # 金色の境界線でトリミング
    trimmed_image = image.crop(bbox)

    # トリミングされた画像を保存
    trimmed_image.save(output_path)

In [ ]:
def batch_process_images(input_folder, output_folder):
    # 出力フォルダが存在しない場合は作成する
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 入力フォルダ内の画像ファイルを処理する
    for filename in os.listdir(input_folder):
        if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.jpeg'):
            input_image_path = os.path.join(input_folder, filename)
            output_image_path = os.path.join(output_folder, filename)
            # 画像のトリミングと保存を実行
            trim_color_paper(input_image_path, output_image_path)

In [ ]:
input_folder_path = "/content/drive/MyDrive/yosetty/input_folder/"  # 入力フォルダのパス
output_folder_path = "/content/drive/MyDrive/yosetty/output_folder/"  # 出力フォルダのパス
batch_process_images(input_folder_path, output_folder_path)